In [1]:
using DataFrames
using CSVFiles
using StatsBase
using LinearAlgebra
import ExcelFiles

In [2]:
function min_max_scale(x)
    dt = fit(UnitRangeTransform, x, dims=1)
    StatsBase.transform(dt, x)
end

data = load("processed_data_5f34929a.csv")|>DataFrame
rename!(data, :sales=>:Sales,
    Symbol("Customer Id")=>:ID,
    :GR=>:Growth,
)
# data = data[!,["Province","City","ID",
#                "Customer Name","Potential","Sales",
#                "ALL_CUST","MA_CUST",
#                "VR_CUST","VR_CALL_CNT","VEEVA_CNT",
#                "SPK_CNT","EMEET_CNT","ADD_WECHAT_CNT",
#                "sales_half_yr_pre","sales_half_yr","hospital_segment","Group"]]

data[!,:Growth] = data.sales_half_yr./data.sales_half_yr_pre.-1
data[!,:Growth] = round.(data.Growth,digits=1)
data[!,:rounded_sales] = round.(data.sales_half_yr,sigdigits=1,base=2)
# replace!(data[!,:Growth],NaN=>0,Inf=>1)
sort!(data, [:Province,:hospital_segment,:Growth,:rounded_sales,:Group]);
# data = data[!,Not(:rounded_sales)]

In [4]:
function initiate(data)
    n = nrow(data)
    n_partition = 3
    m = nrow(data) % 3
    partition = repeat(collect(1:3), n ÷ 3)
    append!(partition, collect(1:3)[begin:m]);
    (n,n_partition,partition)
end 

initiate (generic function with 1 method)

In [5]:
function objective(partition, data, n_partition, n=3)
    group_sales = [data.sales_half_yr[partition .== i]|>sum for i ∈ 1:n_partition] |> std
    group_potential = [data.Potential[partition .== i]|>sum for i ∈ 1:n_partition] |> std
    group_cust_counts = [data.ALL_CUST[partition .== i]|>sum for i ∈ 1:n_partition] |> std
    present = [data.sales_half_yr[partition .== i]|>sum for i ∈ 1:n_partition]
    ex = [data.sales_half_yr_pre[partition .== i]|>sum for i ∈ 1:n_partition]
    group_growth = present ./ ex .-1 |> std
    top_plus_std = [(partition.==i) .& (data.hospital_segment.=="Top Plus") |> sum for i ∈ 1:n_partition] |> std
    [group_sales,group_potential,group_cust_counts,group_growth,top_plus_std]
end 

objective (generic function with 2 methods)

In [6]:
function find_partition(data)
    n,n_partition,partition = initiate(data)
    ϵ = objective(partition, data, n_partition)
    ex_ϵ = ϵ
    
    total_no_of_improvment = 0
    for t ∈ 1:10
        no_of_improvment = 0
        for loc1 ∈ 1:n
            for loc2 ∈ loc1:n

                a1 = partition[loc1]; a2 = partition[loc2]
                    partition[loc1] = a2; partition[loc2] = a1

                    ϵ′ = objective(partition, data, n_partition)
                    if ϵ′[1] <= ϵ[1] && ϵ′[2] <= ϵ[2] && ϵ′[3] <= ϵ[3] && ϵ′[4] <= ϵ[4]
                        ϵ = ϵ′
                        no_of_improvment = no_of_improvment + 1
                    else
                        partition[loc1] = a1; partition[loc2] = a2
                end
            end
        end
        total_no_of_improvment = total_no_of_improvment + no_of_improvment
        println("Number of improvments: $(total_no_of_improvment)")
        println("Best objective value: $(ϵ)")
        println("Ex-objective value: $(ex_ϵ)")
        norm_difference = norm(ex_ϵ.- ϵ)
        println("Norm difference: $(norm_difference)")
        (no_of_improvment == 0 || norm_difference <= 1e-2) && break
        ex_ϵ=ϵ
    end
    println("Total Number of improvments: $(total_no_of_improvment)")
    println("################################################")
    
    total_no_of_tweaks=0
    for t ∈ 1:10
        no_of_tweaks = 0
        for loc ∈ 1:n
            original = partition[loc]
            options = setdiff(1:3,original)
                for option ∈ options
                    partition[loc] = option
                    ϵ′ = objective(partition, data, n_partition)
                    if ϵ′[1] <= ϵ[1] && ϵ′[2] <= ϵ[2] && ϵ′[3] <= ϵ[3] && ϵ′[4] <= ϵ[4]
                        ϵ = ϵ′
                        no_of_tweaks = no_of_tweaks + 1
                    else
                        partition[loc] = original
                end
            end
        end
        total_no_of_tweaks = total_no_of_tweaks + no_of_tweaks
        println("Number of tweaks: $(total_no_of_tweaks)")
        println("Best objective value: $(ϵ)")
        println("Ex-objective value: $(ex_ϵ)")
        norm_difference = norm(ex_ϵ.- ϵ)
        println("Norm difference: $(norm_difference)")
        (no_of_tweaks == 0 || norm_difference <= 1e-2) && break
        ex_ϵ=ϵ
    end
    println("Total Number of tweaks: $(total_no_of_tweaks)")
    println("################################################")
    
    return partition
end

find_partition (generic function with 1 method)

In [7]:
combine(groupby(data,:Group),nrow)

,Group,nrow
,String,Int64
1,LH,819
2,HL,496
3,LL,131
4,HH,265


In [8]:
groups = data.Group|>sort|>unique
res = DataFrame()

for group ∈ groups
    selected = data[data.Group.==group,:]
    println("##############################################")
    println("Optimizing for $group")
    group == "HL" && sort!(selected, [:Province,:hospital_segment,:rounded_sales,:Growth]);
    group == "LH"&& sort!(selected, [:Growth,:rounded_sales]);
    partition=find_partition(selected)
    insertcols!(selected,1,(:Partition=>partition));
    res = vcat(res,selected)
end

##############################################
Optimizing for HH
Number of improvments: 11868
Best objective value: [47052.65803548603, 555.10648956469, 9.237604307034012, 0.0008162592710123664, 0.5773502691896258]
Ex-objective value: [545822.7506721411, 8.206513213254876e6, 42.335957923889396, 0.05969242220930142, 0.5773502691896258]
Norm difference: 8.221102119326315e6
Number of improvments: 23713
Best objective value: [46933.00091987212, 246.31135738281844, 9.237604307034012, 0.0007317016945834418, 0.5773502691896258]
Ex-objective value: [47052.65803548603, 555.10648956469, 9.237604307034012, 0.0008162592710123664, 0.5773502691896258]
Norm difference: 331.16802227308386
Number of improvments: 35550
Best objective value: [46933.00091987212, 246.31135738281844, 9.237604307034012, 0.0007317016945834418, 0.5773502691896258]
Ex-objective value: [46933.00091987212, 246.31135738281844, 9.237604307034012, 0.0007317016945834418, 0.5773502691896258]
Norm difference: 0.0
Total Number of improv

In [10]:
res = res[!,Not(:rounded_sales)];
ExcelFiles.save("Partitioned.xlsx",res)

In [19]:
cols = ["Partition","Province","City","ID",
        "Customer Name","Potential","Sales",
        "ALL_CUST","MA_CUST",
        "VR_CUST","VR_CALL_CNT","VEEVA_CNT",
        "SPK_CNT","EMEET_CNT","ADD_WECHAT_CNT",
        "sales_half_yr_pre","sales_half_yr","hospital_segment","Group"];
res = res[!,cols];

In [20]:
stats = combine(groupby(res,[:Group,:Partition]),AsTable([:sales_half_yr, :sales_half_yr_pre]) => (x -> sum(x.sales_half_yr)/sum(x.sales_half_yr_pre)-1)=>Symbol("Growth"),names(res)[6:end-3].=>sum,nrow=>:Hospital_Counts)
stats = sort!(stats,[:Group,:Partition])
ExcelFiles.save("Overall_Stats.xlsx",stats)

In [24]:
segs = sort!(combine(groupby(res,[:Group,:Partition,:hospital_segment]),nrow),[:Group,:hospital_segment,:Partition])
unstack(segs, [:Group, :Partition], :hospital_segment, :nrow, allowduplicates=true)
ExcelFiles.save("Tier_Counts.xlsx",segs)
# combine(groupby(segs,[:Group,:hospital_segment]),:nrow=>std)

In [ ]:
provincial = sort(combine(groupby(res,[:Group,:Partition,:Province]),nrow),[:Province,:Partition])
provincial
#unstack(provincial, [:Group, :Partition], :Province, :nrow, allowduplicates=true)

In [ ]:
combine(groupby(provincial,:Province),:nrow=>std)[!,2]|>std
combine(groupby(segs,[:Group,:hospital_segment]),:nrow=>std)